In [12]:
import numpy as np
import pandas as pd

In [13]:
ease = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/1_EASE_epoch10_@.csv')
sas = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/SASRec_@.csv')

In [19]:
ngcf = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/NGCF.csv')


In [22]:
gru4rec = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/GRU4Rec.csv')


In [23]:
gru4rec.shape

(311197, 2)

In [21]:
ngcf.shape

(313600, 2)

In [28]:
sas.shape

(311471, 2)

In [24]:
gru4rec.groupby(['user']).count()

,item
user,
11,10
14,10
18,10
25,10
31,10
...,...
138473,10
138475,10
138486,5


In [14]:
sas_user_count=sas.groupby(['user']).count()

In [4]:
sas[sas['user']==11]


,user,item
0,11,48780
1,11,8961
2,11,44665
3,11,7438
4,11,4886
5,11,32587
6,11,3996
7,11,293


In [14]:
ease['prior'] = [i for i in range(8)]*31360
ease.head(10)

,user,item,prior
0,11,4886,0
1,11,4370,1
2,11,8961,2
3,11,40815,3
4,11,2329,4
5,11,32587,5
6,11,37386,6
7,11,47,7
10,14,1198,0
11,14,2011,1


In [4]:
ease8 = ease.groupby('user').head(8)
ease8.head(10)

,user,item
0,11,4886
1,11,4370
2,11,8961
3,11,40815
4,11,2329
5,11,32587
6,11,37386
7,11,47
10,14,1198
11,14,2011


In [5]:
bert['prior'] = [i for i in range(8,18)]*31360

In [6]:
result = pd.concat([ease8, bert])
result

,user,item,prior
0,11,4886,NaN
1,11,4370,NaN
2,11,8961,NaN
3,11,40815,NaN
4,11,2329,NaN
...,...,...,...
313595,138493,593,13.0
313596,138493,4848,14.0
313597,138493,4022,15.0
313598,138493,53125,16.0


In [7]:
result = result.drop_duplicates(['user','item'],keep='first')
result

,user,item,prior
0,11,4886,NaN
1,11,4370,NaN
2,11,8961,NaN
3,11,40815,NaN
4,11,2329,NaN
...,...,...,...
313589,138492,1210,17.0
313592,138493,293,10.0
313596,138493,4848,14.0
313597,138493,4022,15.0


In [8]:
result = result.sort_values(['user','prior'])

In [9]:
result = result.groupby('user').head(10)
result

,user,item,prior
4,11,1704,12.0
6,11,3996,14.0
8,11,1127,16.0
0,11,4886,NaN
1,11,4370,NaN
...,...,...,...
313591,138493,2628,NaN
313592,138493,593,NaN
313593,138493,110,NaN
313594,138493,1270,NaN


In [ ]:
result[['user','item']].to_csv('./output/ratio_ensemble.csv',index=False)

In [3]:
import sys
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import argparse
import os
import glob
import pdb

In [31]:


file_list=['/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/1_EASE_epoch10_@.csv',
           '/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/2_EASE_newversion.csv']




In [27]:
ratios = input(f"(모델 수 : {len(file_list)})띄어쓰기로 구분하여 ratio 입력하기")
ratios = ratios.split(" ")
ratios = list(map(int,ratios))


In [28]:
ratios

[3, 8]

In [30]:
pwd

'/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general'

In [55]:
ratios = input(f"(모델 수 : {len(file_list)})띄어쓰기로 구분하여 ratio 입력하기")
ratios = ratios.split(" ")
ratios = list(map(int,ratios))
if len(ratios) != len(file_list):
    raise ValueError("모델 수와 가중치값의 수가 맞지 않음") 
df_list = [pd.read_csv(f) for f in file_list]

for df,ratio in zip(df_list,ratios):
    df["weight"] = ratio
result = pd.concat(df_list).reset_index(drop=True)
g = result.groupby(['user','item']).agg({'weight': 'sum'})
g['weight'] = - g['weight']
g = g.sort_values(['user','weight'])
g['weight'] = - g['weight']
result = g.groupby('user').head(10)
result = result.reset_index()

,user,item,weight
0,11,47,11
1,11,3996,11
2,11,4370,11
3,11,4886,11
4,11,8961,11
...,...,...,...
313595,138493,5349,11
313596,138493,293,8
313597,138493,4022,8
313598,138493,4848,8


In [58]:
result = result.reset_index()


In [59]:
result[['user','item']]

,user,item
0,11,47
1,11,3996
2,11,4370
3,11,4886
4,11,8961
...,...,...
313595,138493,5349
313596,138493,293
313597,138493,4022
313598,138493,4848


In [15]:

ratios = input(f"(모델 수 : {len(file_list)})띄어쓰기로 구분하여 ratio 입력하기")
ratios = ratios.split(" ")
ratios = list(map(int,ratios))
if len(ratios) != len(file_list):
    raise ValueError("모델 수와 가중치값의 수가 맞지 않음") 
df_list = [pd.read_csv(f) for f in file_list]

for df,ratio in zip(df_list,ratios):
    df["weight"] = ratio
result = pd.concat(df_list).reset_index(drop=True)
g = result.groupby('user').agg({'weight': 'sum', 'item': lambda x: x[1]})

g['weight'] = - g['weight']
g = g.sort_values(['user','weight'])
g['weight'] = - g['weight']
result = g.groupby('user').head(10)
result = result.sort_values(by='user')
result

KeyError: 1